In [1]:

import transformer_lens
from transformer_lens import utils
import torch
import tqdm
from functools import partial

/Users/pavan/miniconda3/envs/interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/pavan/miniconda3/envs/interp/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
model = transformer_lens.HookedTransformer.from_pretrained("roneneldan/TinyStories-33M")

logits, activations = model.run_with_cache("Hello World my dear")

Using pad_token, but it is not set yet.


Loaded pretrained model roneneldan/TinyStories-33M into HookedTransformer


In [3]:
layer_to_ablate = 0
head_index_to_ablate = 8

tokens = model.to_tokens("I am going to")
act_vector = torch.rand(5,4)

def head_ablation_hook(
    value,
    hook,
    act_vector
):
    print(f"Shape of the value tensor: {value.shape}")
    value[:, :, head_index_to_ablate, :] = value[:, :, head_index_to_ablate, :] + act_vector
    return value

original_loss = model(tokens, return_type="loss")
ablated_loss = model.run_with_hooks(
    tokens, 
    return_type="loss", 
    fwd_hooks=[(
        utils.get_act_name("v", layer_to_ablate), 
        partial(head_ablation_hook, act_vector=act_vector)
        )]
    )
model.reset_hooks()
print(f"Original loss: {original_loss}")
print(f"Ablated loss: {ablated_loss}")

Shape of the value tensor: torch.Size([1, 5, 16, 48])


RuntimeError: The size of tensor a (48) must match the size of tensor b (4) at non-singleton dimension 2

In [5]:
model.generate("I am going to kill you for", max_new_tokens=100)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:03<00:00, 32.69it/s]


('I am going to kill you for dinner tonight!"\n\nHer friends all looked at her, but they could not see a knife because of her idea for the weapon. They thought it was a toy and not good for hurting someone. Lily tried one more time to teach them an idea.\n\nShe whispered, "I am serious, so we must never touch that weapon. It\'s not something we can do." Her friends nodded in agreement and they all went inside to eat dinner.\n\nBut later that day, Lily got',
 tensor([[ 8.4708,  9.4165, -1.5127,  ..., -1.4795,  0.1042,  3.6015],
         [18.2417, 14.2405,  0.5199,  ..., -1.8031,  2.5240,  4.5438],
         [20.8377, 17.3307,  1.4376,  ..., -1.7988,  0.2627,  3.2493],
         ...,
         [10.2109,  7.2637,  1.7724,  ..., -1.0375,  1.9692,  3.0924],
         [ 3.0303,  3.7021, -1.6846,  ..., -0.8582,  1.5898,  4.0705],
         [ 7.7399,  4.3363, -1.8939,  ..., -0.9965, 11.2535,  4.2337]],
        device='mps:0'),
 tensor([[50256,    40,   716,  1016,   284,  1494,   345,   329,  8073,